In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import time

In [2]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, plot_precision_recall_curve
from sklearn.metrics import precision_score,recall_score,f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
from sklearn.multiclass import OneVsRestClassifier

In [7]:
from sklearn.linear_model import LogisticRegression

In [8]:
from sklearn.model_selection import GridSearchCV

In [9]:
df = pd.read_csv("vg_sample.csv")
df.head()

,Unnamed: 0.1,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,title,brand,main_cat,Polarity,Analysis
0,434179,434435,1.0,True,"05 7, 2015",A1GF8Y5Z6HRYM9,B0000D9MR2,Kristy,picture shows jacks separate cables. item rece...,ripoff.,1430956800,universal rfu adapter,by intec,video games,0.000000,Neutral
1,1464590,1465462,1.0,True,"10 11, 2015",A1BS5IMS64EAR4,B006ZNGSFW,CHARLES MELTON,good,one star,1444521600,hoyle card games download,by encore,video games,0.700000,Positive
2,1771737,1772883,1.0,True,"12 12, 2014",AUFFVX36M1AIH,B00DTWEOZ8,Andy L.,bland fps shooter. ...,one star,1418342400,titanfall online game code,by electronic arts,video games,-0.166667,Negative
3,554058,554376,1.0,False,"01 24, 2006",A1WCOALU7PAZHC,B0009WPZPE,Mark Twain,"worst series, newest system mean best.",doa4,1138060800,dead or alive,by tecmo koei,video games,-0.104167,Negative
4,2375119,2377226,1.0,True,"10 9, 2016",AVLKM2GMOP8E4,B00ZQB28XK,Murdock,"seen coming promises spore, sense disappointme...",it happened again...,1475971200,no man s sky playstation,by sony,video games,-0.187500,Negative


In [10]:
df1 = df.copy()

In [11]:
df1 = df1[df1.Analysis != 'Neutral']

In [12]:
en = LabelEncoder()
df1['Analysis_1'] = en.fit_transform(df1['Analysis'])

In [56]:
df1.head()

,Unnamed: 0.1,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,title,brand,main_cat,Polarity,Analysis,Analysis_1
1,1464590,1465462,1.0,True,"10 11, 2015",A1BS5IMS64EAR4,B006ZNGSFW,CHARLES MELTON,good,one star,1444521600,hoyle card games download,by encore,video games,0.700000,Positive,1
2,1771737,1772883,1.0,True,"12 12, 2014",AUFFVX36M1AIH,B00DTWEOZ8,Andy L.,bland fps shooter. ...,one star,1418342400,titanfall online game code,by electronic arts,video games,-0.166667,Negative,0
3,554058,554376,1.0,False,"01 24, 2006",A1WCOALU7PAZHC,B0009WPZPE,Mark Twain,"worst series, newest system mean best.",doa4,1138060800,dead or alive,by tecmo koei,video games,-0.104167,Negative,0
4,2375119,2377226,1.0,True,"10 9, 2016",AVLKM2GMOP8E4,B00ZQB28XK,Murdock,"seen coming promises spore, sense disappointme...",it happened again...,1475971200,no man s sky playstation,by sony,video games,-0.187500,Negative,0
5,2815274,2818093,1.0,True,"11 18, 2016",A12ZXNUP2ZIN1S,B00TXO703Y,Tyrone Scott,1. 4 face buttons tabs original buttons button...,1. 4 face buttons don't have all the tabs ...,1479427200,modfreakz trade button set dpad share chrome g...,by mod freakz,video games,0.050529,Positive,1


In [43]:
df1.dropna(inplace = True)

In [44]:
df1.isnull().sum()

Unnamed: 0.1      0
Unnamed: 0        0
overall           0
verified          0
reviewTime        0
reviewerID        0
asin              0
reviewerName      0
reviewText        0
summary           0
unixReviewTime    0
title             0
brand             0
main_cat          0
Polarity          0
Analysis          0
dtype: int64

In [45]:
model_accuracy=[]
model_precision=[]
model_recall=[]
model_f1_score=[]
model_roc_auc_score=[]
model_train_time=[]

In [46]:
# making a function to train the model and evaluate the model
def train_and_score (model,X_test,X_train,y_test,y_train, best_parameter=True, best_score=True):
    #storing the start time of model training
    start=time.time()
    #training the model
    #model.fit(X_train,y_train)
    model = OneVsRestClassifier(model)
    model.fit(X_train,y_train)
    #storing the time when the model ended trining 
    stop = time.time()
      #calculating the total time taken for model training 
    time_min=round((stop - start)/60,4)
      # printing the time taken to train the model
    print(f"Training time: {time_min}min")
    model_train_time.append(time_min)
  
  

  # finding the best parameters of the model if any 
    if best_parameter==True:
        
        # using try except method to avoid any error
        try:
            print("The best parameters found out to be :" ,model.best_params_)
        except:
            pass
        
        
  # finding best score of the model 
    if best_score==True:
        try:
            print(" \nwhere negative mean squared error is: ", model.best_score_,'\n')
        except:
            pass


  #predicting the values of y from x via model
    y_pred_test = model.predict(X_test)
    y_pred_train = model.predict(X_train)

    def score (actual,predicted,append=True):
        # calculating accuracy score
        accuracy = accuracy_score(actual,predicted)
        # calculating precision score
        precision = precision_score(actual,predicted)
        # calculating recall score
        recall = recall_score(actual,predicted, average = 'weighted')
        # calculating f1_score score
        f1 = f1_score(actual,predicted, average = 'weighted')
        # calculating roc_auc_score score
        roc= roc_auc_score(actual,predicted, average = 'weighted')
    
        # printing all the scores
        print("\tThe accuracy is ", accuracy)
        print("\tThe precision is ", precision)
        print("\tThe recall is ", recall)
        print("\tThe f1 is ", f1)
        print('\tthe auc  is ',roc,'\n')
        print('\t\t\t\tclassification report\n','*'*80,'\n',classification_report(actual,predicted))
    

    # uploading all the score of the matrix in in the list 
        if append == True:
            model_accuracy.append(accuracy)
            model_precision.append(precision)
            model_recall.append(recall)
            model_f1_score.append(f1)
            model_roc_auc_score.append(roc)
        else:
            pass
    # printing the score by the help of coustom score function
    # for train set
    print('\t\t\t\tscore matrix for train')
    print('*'*80)
    score(y_train,y_pred_train,False)
    print('\n')
    # for test set
    print('\t\t\t\tscore matrix for test')
    print('*'*80)
    score(y_test,y_pred_test)

In [62]:
x = df1['reviewText']
y = df1['Analysis_1']

In [63]:
X_train,X_test,y_train,y_test = train_test_split(x,y, test_size = 0.25, stratify=df1['Analysis_1'])

In [64]:
vectorizer = TfidfVectorizer(stop_words="english",min_df=0.01,max_df=0.95)
X_train = vectorizer.fit_transform(X_train)
X_test =vectorizer.transform(X_test)

# Multinomial NB

In [65]:
from sklearn.naive_bayes import MultinomialNB
# creating MultinomialNB model
model = MultinomialNB()

In [66]:
# training and evaluating the MultinomialNB
train_and_score(model,X_test,X_train,y_test,y_train)

Training time: 0.0029min
				score matrix for train
********************************************************************************
	The accuracy is  0.8189129694307604
	The precision is  0.8154180895902506
	The recall is  0.8189129694307604
	The f1 is  0.7626145566106345
	the auc  is  0.5724972880680518 

				classification report
 ******************************************************************************** 
               precision    recall  f1-score   support

           0       0.92      0.15      0.26     82390
           1       0.82      1.00      0.90    311012

    accuracy                           0.82    393402
   macro avg       0.87      0.57      0.58    393402
weighted avg       0.84      0.82      0.76    393402



				score matrix for test
********************************************************************************
	The accuracy is  0.8187121570302134
	The precision is  0.8152775256690528
	The recall is  0.8187121570302134
	The f1 is  0.7622890297992251
	the

# lightGBM

In [26]:
import lightgbm as lgb

In [74]:
lgbc=lgb.LGBMClassifier()

lgbc_para={'n_estimators': [100,125,150],'max_depth': [15,20,25]}
lgbc_grid=GridSearchCV(estimator=lgbc,param_grid=lgbc_para, cv=3,scoring='recall',verbose=5,n_jobs=-1)

In [75]:
train_and_score(lgbc_grid,X_test, X_train, y_test, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Training time: 2.9497min
				score matrix for train
********************************************************************************
	The accuracy is  0.9069043878780484
	The precision is  0.9177572882346852
	The recall is  0.9069043878780484
	The f1 is  0.9026772059389879
	the auc  is  0.8206329420469759 

				classification report
 ******************************************************************************** 
               precision    recall  f1-score   support

           0       0.85      0.67      0.75     82390
           1       0.92      0.97      0.94    311012

    accuracy                           0.91    393402
   macro avg       0.88      0.82      0.85    393402
weighted avg       0.90      0.91      0.90    393402



				score matrix for test
********************************************************************************
	The accuracy is  0.90295423002425
	The precision is  0.914625695267621
	The recall i

# Decision Tree

In [67]:
from sklearn.tree import DecisionTreeClassifier
# creating DecisionTree model
DecisionTree = DecisionTreeClassifier(max_depth=10,max_leaf_nodes=45,criterion='entropy')

# training and evaluating the DecisionTree
train_and_score(DecisionTree,X_test, X_train, y_test, y_train)

Training time: 0.1706min
				score matrix for train
********************************************************************************
	The accuracy is  0.8391645187365595
	The precision is  0.8760287175750953
	The recall is  0.8391645187365595
	The f1 is  0.8313649799566329
	the auc  is  0.7160987849122159 

				classification report
 ******************************************************************************** 
               precision    recall  f1-score   support

           0       0.65      0.50      0.57     82390
           1       0.88      0.93      0.90    311012

    accuracy                           0.84    393402
   macro avg       0.76      0.72      0.73    393402
weighted avg       0.83      0.84      0.83    393402



				score matrix for test
********************************************************************************
	The accuracy is  0.8396449433403999
	The precision is  0.8760009463492757
	The recall is  0.8396449433403999
	The f1 is  0.8317325348006253
	the

# XGBoost

In [78]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 70.9/70.9 MB 4.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [79]:
from xgboost import XGBClassifier

In [81]:
xgbc=XGBClassifier()

In [82]:
xgbc_param={'n_estimators': [100,125,150],'max_depth': [7,10,15],'criterion': ['entropy']}
xgbc_grid=GridSearchCV(estimator=xgbc,param_grid=xgbc_param,cv=3,scoring='recall',verbose=5,n_jobs=-1)

In [ ]:
train_and_score(xgbc_grid,X_test, X_train, y_test, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


In [17]:
df['Analysis_1'] = en.fit_transform(df['Analysis'])

In [18]:
df.head()

,Unnamed: 0.1,Unnamed: 0,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,title,brand,main_cat,Polarity,Analysis,Analysis_1
0,434179,434435,1.0,True,"05 7, 2015",A1GF8Y5Z6HRYM9,B0000D9MR2,Kristy,picture shows jacks separate cables. item rece...,ripoff.,1430956800,universal rfu adapter,by intec,video games,0.000000,Neutral,1
1,1464590,1465462,1.0,True,"10 11, 2015",A1BS5IMS64EAR4,B006ZNGSFW,CHARLES MELTON,good,one star,1444521600,hoyle card games download,by encore,video games,0.700000,Positive,2
2,1771737,1772883,1.0,True,"12 12, 2014",AUFFVX36M1AIH,B00DTWEOZ8,Andy L.,bland fps shooter. ...,one star,1418342400,titanfall online game code,by electronic arts,video games,-0.166667,Negative,0
3,554058,554376,1.0,False,"01 24, 2006",A1WCOALU7PAZHC,B0009WPZPE,Mark Twain,"worst series, newest system mean best.",doa4,1138060800,dead or alive,by tecmo koei,video games,-0.104167,Negative,0
4,2375119,2377226,1.0,True,"10 9, 2016",AVLKM2GMOP8E4,B00ZQB28XK,Murdock,"seen coming promises spore, sense disappointme...",it happened again...,1475971200,no man s sky playstation,by sony,video games,-0.187500,Negative,0


In [19]:
df.dropna(inplace = True)

# LOGISTIC REGRESSION

In [20]:
x = df['reviewText']
y = df['Analysis']

In [21]:
cv = TfidfVectorizer()
x = cv.fit_transform(x)

In [22]:
X_train,X_test,y_train,y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

In [32]:
t=time.time()
lr1=LogisticRegression(max_iter=100)
ovr1=OneVsRestClassifier(lr1)
ovr1.fit(X_train,y_train)

y_pred_train = ovr1.predict(X_train)
y_pred_test = ovr1.predict(X_test)

end=time.time()-t

print(end)

34.16789484024048


In [35]:
print(classification_report(y_pred_train, y_train))

              precision    recall  f1-score   support

    Negative       0.90      0.95      0.92     82198
     Neutral       0.91      0.96      0.94     32812
    Positive       0.99      0.97      0.98    335678

    accuracy                           0.96    450688
   macro avg       0.93      0.96      0.95    450688
weighted avg       0.97      0.96      0.96    450688



In [36]:
print(classification_report(y_pred_test, y_test))

              precision    recall  f1-score   support

    Negative       0.87      0.94      0.91     20448
     Neutral       0.90      0.95      0.92      8139
    Positive       0.98      0.96      0.97     84086

    accuracy                           0.96    112673
   macro avg       0.92      0.95      0.93    112673
weighted avg       0.96      0.96      0.96    112673



In [56]:
print('\t\tscore matrix for train')
print('*'*80)

print("Accuracy Score :", accuracy_score(y_pred_train, y_train))
print("Precision Score :", precision_score(y_pred_train, y_train, average = 'weighted'))
print("F1 Score Score :", f1_score(y_pred_train, y_train, average = 'weighted'))
print("Recall Score :", recall_score(y_pred_train, y_train, average = 'weighted'))



print('\t\tclassification report\n','*'*80,'\n',classification_report(y_pred_train, y_train))

		score matrix for train
********************************************************************************
Accuracy Score : 0.9642058364101108
Precision Score : 0.9654933016567261
F1 Score Score : 0.9645906976861628
Recall Score : 0.9642058364101108
		classification report
 ******************************************************************************** 
               precision    recall  f1-score   support

    Negative       0.90      0.95      0.92     82198
     Neutral       0.91      0.96      0.94     32812
    Positive       0.99      0.97      0.98    335678

    accuracy                           0.96    450688
   macro avg       0.93      0.96      0.95    450688
weighted avg       0.97      0.96      0.96    450688



In [57]:
print('\t\tscore matrix for test')
print('*'*80)

print("Accuracy Score :", accuracy_score(y_pred_test, y_test))
print("Precision Score :", precision_score(y_pred_test, y_test, average = 'weighted'))
print("F1 Score Score :", f1_score(y_pred_test, y_test, average = 'weighted'))
print("Recall Score :", recall_score(y_pred_test, y_test, average = 'weighted'))



print('\t\tclassification report\n','*'*80,'\n', classification_report(y_pred_test, y_test))

		score matrix for test
********************************************************************************
Accuracy Score : 0.9563426907954878
Precision Score : 0.9581057799728819
F1 Score Score : 0.9568846872761685
Recall Score : 0.9563426907954878
		classification report
 ******************************************************************************** 
               precision    recall  f1-score   support

    Negative       0.87      0.94      0.91     20448
     Neutral       0.90      0.95      0.92      8139
    Positive       0.98      0.96      0.97     84086

    accuracy                           0.96    112673
   macro avg       0.92      0.95      0.93    112673
weighted avg       0.96      0.96      0.96    112673



In [61]:
dic0 = pd.DataFrame({"Actual" : y_test, "Predicted" : y_pred_test})
dic0[:31]

,Actual,Predicted
112252,Positive,Positive
198160,Positive,Positive
361464,Positive,Positive
554755,Positive,Positive
344849,Positive,Positive
401232,Positive,Positive
303450,Positive,Positive
218360,Neutral,Neutral
170458,Positive,Positive
22604,Negative,Negative


# LIGHTGBM

In [37]:
t = time.time()
lgbm = lgb.LGBMClassifier()
ovr2 = OneVsRestClassifier(lgbm)
ovr2.fit(X_train, y_train)

y_pred_train2 = ovr2.predict(X_train)
y_pred_test2 = ovr2.predict(X_test)

end = time.time()-t

print(end)

213.47250056266785


In [38]:
print(classification_report(y_pred_train2, y_train))

              precision    recall  f1-score   support

    Negative       0.74      0.89      0.81     72461
     Neutral       0.93      0.83      0.88     38653
    Positive       0.97      0.94      0.95    339574

    accuracy                           0.92    450688
   macro avg       0.88      0.89      0.88    450688
weighted avg       0.93      0.92      0.92    450688



In [53]:
print(classification_report(y_pred_test2, y_test))

              precision    recall  f1-score   support

    Negative       0.72      0.89      0.80     17911
     Neutral       0.93      0.83      0.88      9622
    Positive       0.97      0.93      0.95     85140

    accuracy                           0.92    112673
   macro avg       0.87      0.88      0.87    112673
weighted avg       0.93      0.92      0.92    112673



In [58]:
print('\t\tscore matrix for train')
print('*'*80)

print("Accuracy Score :", accuracy_score(y_pred_train2, y_train))
print("Precision Score :", precision_score(y_pred_train2, y_train, average = 'weighted'))
print("F1 Score Score :", f1_score(y_pred_train2, y_train, average = 'weighted'))
print("Recall Score :", recall_score(y_pred_train2, y_train, average = 'weighted'))



print('\t\tclassification report\n','*'*80,'\n',classification_report(y_pred_train2, y_train))

		score matrix for train
********************************************************************************
Accuracy Score : 0.9207544909116728
Precision Score : 0.9277982723831774
F1 Score Score : 0.9227642794120378
Recall Score : 0.9207544909116728
		classification report
 ******************************************************************************** 
               precision    recall  f1-score   support

    Negative       0.74      0.89      0.81     72461
     Neutral       0.93      0.83      0.88     38653
    Positive       0.97      0.94      0.95    339574

    accuracy                           0.92    450688
   macro avg       0.88      0.89      0.88    450688
weighted avg       0.93      0.92      0.92    450688



In [60]:
print('\t\tscore matrix for test')
print('*'*80)

print("Accuracy Score :", accuracy_score(y_pred_test2, y_test))
print("Precision Score :", precision_score(y_pred_test2, y_test, average = 'weighted'))
print("F1 Score Score :", f1_score(y_pred_test2, y_test, average = 'weighted'))
print("Recall Score :", recall_score(y_pred_test2, y_test, average = 'weighted'))



print('\t\tclassification report\n','*'*80,'\n', classification_report(y_pred_test2, y_test))

		score matrix for test
********************************************************************************
Accuracy Score : 0.916954372387351
Precision Score : 0.9252640768064708
F1 Score Score : 0.919337452527188
Recall Score : 0.916954372387351
		classification report
 ******************************************************************************** 
               precision    recall  f1-score   support

    Negative       0.72      0.89      0.80     17911
     Neutral       0.93      0.83      0.88      9622
    Positive       0.97      0.93      0.95     85140

    accuracy                           0.92    112673
   macro avg       0.87      0.88      0.87    112673
weighted avg       0.93      0.92      0.92    112673



In [62]:
dic = pd.DataFrame({"Actual" : y_test, "Predicted" : y_pred_test2})
dic[:31]

,Actual,Predicted
112252,Positive,Positive
198160,Positive,Positive
361464,Positive,Positive
554755,Positive,Positive
344849,Positive,Positive
401232,Positive,Positive
303450,Positive,Positive
218360,Neutral,Neutral
170458,Positive,Positive
22604,Negative,Negative
